In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("memento_data/memento_data_20250419_004034.csv")

In [2]:
%pip install langdetect

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

def is_english(text):
    try:
        return detect(str(text)) == "en"
    except LangDetectException:
        return False

# Remove duplicate link (OpenAI weirdly have duplicate link for different language)
duplicates = df[df.duplicated("link", keep=False)].copy()
duplicates["is_english"] = duplicates["description"].apply(is_english)
english_duplicates = duplicates[duplicates["is_english"]].drop(columns="is_english")
df_cleaned = df[~df["link"].isin(duplicates[~duplicates["is_english"]]["link"])]
df_cleaned.drop_duplicates(subset="link", keep="first", inplace=True)
print(df_cleaned)

                                        source  \
0      https://machinelearningmastery.com/feed   
1      https://machinelearningmastery.com/feed   
2      https://machinelearningmastery.com/feed   
3      https://machinelearningmastery.com/feed   
4      https://machinelearningmastery.com/feed   
...                                        ...   
26645   http://proceedings.mlr.press//feed.xml   
26646   http://proceedings.mlr.press//feed.xml   
26647   http://proceedings.mlr.press//feed.xml   
26648   http://proceedings.mlr.press//feed.xml   
26649   http://proceedings.mlr.press//feed.xml   

                                  channel_title  \
0                    MachineLearningMastery.com   
1                    MachineLearningMastery.com   
2                    MachineLearningMastery.com   
3                    MachineLearningMastery.com   
4                    MachineLearningMastery.com   
...                                         ...   
26645  Proceedings of Machine Learning Res

C:\Users\Tan Vo\AppData\Local\Temp\ipykernel_34448\2585802981.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop_duplicates(subset="link", keep="first", inplace=True)


In [4]:
%pip install aiohttp brotli

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

In [16]:
import aiohttp
import pandas as pd
from aiohttp import ClientSession, ClientTimeout
from aiohttp.client_exceptions import ClientError
from tqdm.notebook import tqdm
import yarl
import ssl

# Quick and dirty fix for SSL errors
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:138.0) Gecko/20100101 Firefox/138.0",
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
#     "Accept-Language": "en-US,en;q=0.5",
#     "Accept-Encoding": "gzip, deflate, br, zstd",
#     "DNT": "1",
#     "Sec-GPC": "1",
#     "Connection": "keep-alive",
#     # "Cookie": (
#     #     'advanced_ads_pro_visitor_referrer={"expires":1776496072,"data":"https://github.com/"}; '
#     #     'advanced_ads_page_impressions={"expires":2060320072,"data":11}; '
#     #     'advanced_ads_visitor={"browser_width":1920}; __adblocker=true'
#     # ),
#     "Upgrade-Insecure-Requests": "1",
#     "Sec-Fetch-Dest": "document",
#     "Sec-Fetch-Mode": "navigate",
#     "Sec-Fetch-Site": "none",
#     "Sec-Fetch-User": "?1",
#     "Priority": "u=0, i",
#     "TE": "trailers",
# }
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; ArchiveFetcher/1.0; +https://example.com)"
}
async def fetch(session, url, sem):
    url = yarl.URL(url, encoded=True)
    async with sem:
        try:
            async with session.get(url, headers=headers, ssl=ssl_context) as response:
                if response.status == 200:
                    return await response.text()
                else:
                    print(f"Failed to fetch {url}: {response.status}")
                    return None
        except ClientError as e:
            print(f"Request failed: {e} at {url}")
            return None
        except Exception as e:
            print(f"Unexpected error: {e} at {url}")
            return None

async def scrape_all(urls, concurrency=50):
    timeout = ClientTimeout(total=60)
    sem = asyncio.Semaphore(concurrency)
    async with ClientSession(timeout=timeout) as session:
        tasks = [fetch(session, url, sem) for url in urls]
        return await asyncio.gather(*tasks)

In [7]:
df_cleaned["html"] = None

C:\Users\Tan Vo\AppData\Local\Temp\ipykernel_34448\4227107817.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["html"] = None


In [8]:
html_results = [None] * len(df_cleaned)

In [21]:
urls = df_cleaned["link"].tolist()
batch_size = 500
for i in tqdm(range(0, len(urls), batch_size)):
    batch_urls = urls[i : i + batch_size]
    batch_existing_html = html_results[i : i + batch_size]

    if all(html is not None for html in batch_existing_html):
        continue

    to_scrape = [
        (j, url)
        for j, (html, url) in enumerate(zip(batch_existing_html, batch_urls))
        if html is None
    ]

    if not to_scrape:
        continue

    scrape_indices, scrape_urls = zip(*to_scrape)
    batch_results = await scrape_all(scrape_urls)

    for j, html in zip(scrape_indices, batch_results):
        html_results[i + j] = html

  0%|          | 0/52 [00:00<?, ?it/s]

Failed to fetch https://machinelearningmastery.com/how-does-attention-work-in-encoder-decoder-recurrent-neural-networks/: 403
Failed to fetch https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/: 403
Failed to fetch https://machinelearningmastery.com/implementation-patterns-encoder-decoder-rnn-architecture-attention/: 403
Failed to fetch https://machinelearningmastery.com/building-a-recommender-system-from-scratch-with-matrix-factorization-in-python/: 403
Failed to fetch https://machinelearningmastery.com/implementing-multilingual-translation-with-t5-and-transformers/: 403
Failed to fetch https://machinelearningmastery.com/a-gentle-introduction-to-graph-neural-networks-in-python/: 403
Failed to fetch https://machinelearningmastery.com/building-qa-systems-with-distilbert-and-transformers/: 403
Failed to fetch https://machinelearningmastery.com/the-beginners-guide-to-machine-learning-with-rust/: 403
Failed to fetch https://machinelearningmas

In [22]:
df_cleaned["html"] = html_results

C:\Users\Tan Vo\AppData\Local\Temp\ipykernel_34448\4154457282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["html"] = html_results


In [23]:
import pandas as pd
import os

output_dir = "memento_data"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, f"html_results.csv")
df_cleaned.to_csv(output_file, index=False)
print(f"\nSaved unique items to {output_file}")


Saved unique items to memento_data\html_results.csv
